In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [3]:
class Cell:
    def __init__(self, model_type, params): # where model_type is a string and params is a dictionary with the parameters
        self.model_type = model_type
        self.params = params

    def growth_rate(self, substrate_conc):
        if self.model_type == 'monod':
            return monod(substrate_conc)
        elif self.model_type == 'haldane':
            return haldane(substrate_conc)
        elif self.model_type == 'contois':
            return contois(substrate_conc)
        elif self.model_type == 'moser':
            return moser(substrate_conc)
        elif self.model_type == 'tessier':
            return tessier(substrate_conc)
        elif self.model_type == 'luedeking_piret':
            return luedeking_piret(substrate_conc)
        
        


    def monod(self, s):
        vmax = self.params['vmax']
        km = self.params['km']
        return vmax * s / (km + s)

    def haldane(self, s):
        vmax = self.params['vmax']
        km = self.params['km']
        ki = self.params['ki']
        return vmax * s / (km + s + (s**2 / ki))

    def contois(self, s):
        vmax = self.params['vmax']
        km = self.params['km']
        return vmax * s / (km * x + s)

    def moser(self, s):
        vmax = self.params['vmax']
        km = self.params['km']
        n = self.params['n']
        return vmax * (s**n) / (km + (s**n))

    def tessier(self, s):
        umax = self.params['umax']
        k = self.params['km']
        return umax * (1 - np.exp(-s /km))

    def luedeking_piret(self, s):
        pass



class Tank:
    def __init__(self, initial_biomass, initial_substrate, cell, volume, initial_product = 0):
        self.biomass = initial_biomass
        self.substrate = initial_substrate
        self.product = initial_product
        self.cell = cell
        self.volume = volume

## reaction_rates method

